In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# IMPORTACIÓN DE DATOS

In [2]:
df = pd.read_csv("C:\\Users\\Kim\\Downloads\\RecomendadorEcommerce\\bbdd_ratings.csv")
df.head()

,Id,Title,User_id,review/score,image
0,0,Economics in one lesson,A2FYWUHFF21Q8F,5.0,http://books.google.com/books/content?id=A50iB...
1,0,Economics in one lesson,A5P9PV92PRYEE,5.0,http://books.google.com/books/content?id=A50iB...
2,0,Economics in one lesson,A1D4N3NVIN55PK,4.0,http://books.google.com/books/content?id=A50iB...
3,0,Economics in one lesson,A34AW9TMV7F69T,3.0,http://books.google.com/books/content?id=A50iB...
4,0,Economics in one lesson,A1IHQR1NT6CKVR,5.0,http://books.google.com/books/content?id=A50iB...


In [3]:
df.shape

(1055372, 5)

In [4]:
df.dropna(subset=["image"], inplace=True)  # Elimina NaN en columna image desde el principio
df.shape

(875809, 5)

In [5]:
df.rename(columns={'review/score': 'review'}, inplace=True)  # Renombrar review/score

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875809 entries, 0 to 1054203
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Id       875809 non-null  int64  
 1   Title    875634 non-null  object 
 2   User_id  875809 non-null  object 
 3   review   875809 non-null  float64
 4   image    875809 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 40.1+ MB


In [7]:
df['review'] = df['review'].astype(int)  # Cambiar el tipo de la columna review

# PREPARACIÓN DE DATOS

In [8]:
# Verificación de valores de review y visualización de los primeros datos
print(df['review'].value_counts().sort_index())

1     56402
2     43225
3     75053
4    172537
5    528592
Name: review, dtype: int64


In [9]:
df.head()

,Id,Title,User_id,review,image
0,0,Economics in one lesson,A2FYWUHFF21Q8F,5,http://books.google.com/books/content?id=A50iB...
1,0,Economics in one lesson,A5P9PV92PRYEE,5,http://books.google.com/books/content?id=A50iB...
2,0,Economics in one lesson,A1D4N3NVIN55PK,4,http://books.google.com/books/content?id=A50iB...
3,0,Economics in one lesson,A34AW9TMV7F69T,3,http://books.google.com/books/content?id=A50iB...
4,0,Economics in one lesson,A1IHQR1NT6CKVR,5,http://books.google.com/books/content?id=A50iB...


In [10]:
df= df[:15809]

In [11]:
df["Id"].astype(int)

0         0
1         0
2         0
3         0
4         0
         ..
17975    59
17976    59
17977    59
17978    59
17979    59
Name: Id, Length: 15809, dtype: int32

In [12]:
# Creamos una matriz item-usuario solo con los datos necesarios
iu = df.pivot_table(index='Id', columns='User_id', values='review', fill_value=0)
iu.head()

User_id,A01038432MVI9JXYTTK5T,A0469729ADTHXTW0CPIS,A0503161NJ9GM5VQKEO5,A06650603VOJNJDLFF8N0,A07430931T9PR7QLY3LZR,A09928591YYJUR492WKJA,A101DG7P9E26PW,A101LFK9ZIHMIJ,A101ZR4DTK7P4,A10292Y39VWQDR,...,AZWG3PF80735Q,AZWNCR05NRV9V,AZX7FSZ98E9HV,AZXA75MSDBXO3,AZXFUIA3GYE0E,AZXGPM8EKSHE9,AZY9WNBIVQ983,AZYIHKBD3FZBI,AZYMMVR2C5IQA,AZZQV95X90WT7
Id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
duplicados = df[df[['Id', 'image']].duplicated()]
duplicados

,Id,Title,User_id,review,image
1,0,Economics in one lesson,A5P9PV92PRYEE,5,http://books.google.com/books/content?id=A50iB...
2,0,Economics in one lesson,A1D4N3NVIN55PK,4,http://books.google.com/books/content?id=A50iB...
3,0,Economics in one lesson,A34AW9TMV7F69T,3,http://books.google.com/books/content?id=A50iB...
4,0,Economics in one lesson,A1IHQR1NT6CKVR,5,http://books.google.com/books/content?id=A50iB...
5,0,Economics in one lesson,A3J4GBAN27WS3V,5,http://books.google.com/books/content?id=A50iB...
...,...,...,...,...,...
17975,59,The Other Boleyn Girl,ALDIFQMAWTX6K,5,http://books.google.com/books/content?id=XiRWX...
17976,59,The Other Boleyn Girl,A3IK1QD4XT1ZH9,5,http://books.google.com/books/content?id=XiRWX...
17977,59,The Other Boleyn Girl,A2J7SGZKXBE40E,5,http://books.google.com/books/content?id=XiRWX...
17978,59,The Other Boleyn Girl,AFX7YGMJDALTN,3,http://books.google.com/books/content?id=XiRWX...


In [14]:
# Conservar información de imagen para cada Id
images = df[['Id', 'image']].drop_duplicates().set_index('Id')

In [15]:
images.head()

,image
Id,
0,http://books.google.com/books/content?id=A50iB...
2,http://books.google.com/books/content?id=AeM0A...
3,http://books.google.com/books/content?id=A6Zzg...
4,http://books.google.com/books/content?id=XE_Ot...
6,http://books.google.com/books/content?id=hrJdI...


In [16]:
# CÁLCULO DE LA SIMILITUD DE ITEMS
similitud_items = cosine_similarity(iu.to_numpy())
similitud_items

array([[1.        , 0.        , 0.        , ..., 0.00412042, 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.00132595],
       ...,
       [0.00412042, 0.        , 0.        , ..., 1.        , 0.00978899,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00978899, 1.        ,
        0.00519071],
       [0.        , 0.        , 0.00132595, ..., 0.        , 0.00519071,
        1.        ]])

In [17]:
# CREACIÓN DE UNA MATRIZ DE RECOMENDACIONES
matriz_recomendaciones = pd.DataFrame(similitud_items, index=iu.index, columns=iu.index)
matriz_recomendaciones.head()

Id,0,2,3,4,6,8,9,10,11,12,...,50,51,52,53,54,55,56,57,58,59
Id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.0,0.0,0.0,0.005578,0.001709,0.000000,0.001709,0.003591,0.000000,...,0.0,0.000000,0.000000,0.004346,0.004816,0.001332,0.0,0.004120,0.000000,0.000000
2,0.000000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.008325,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.000000,0.0,1.0,0.0,0.000000,0.005123,0.005784,0.005123,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.006536,0.000000,0.0,0.000000,0.000000,0.001326
4,0.000000,0.0,0.0,1.0,0.000000,0.002621,0.005637,0.002621,0.000000,0.007739,...,0.0,0.000000,0.000000,0.000000,0.002986,0.000000,0.0,0.000000,0.004373,0.000000
6,0.005578,0.0,0.0,0.0,1.000000,0.004842,0.000000,0.004842,0.000000,0.000000,...,0.0,0.014202,0.006897,0.010154,0.005662,0.016980,0.0,0.014879,0.005890,0.000000


In [18]:
# Convertir el DataFrame de una matriz a un formato largo
matriz_recomendaciones_long = matriz_recomendaciones.stack().rename_axis(['id1', 'id2']).reset_index(name='similitud')
matriz_recomendaciones_long

,id1,id2,similitud
0,0,0,1.000000
1,0,2,0.000000
2,0,3,0.000000
3,0,4,0.000000
4,0,6,0.005578
...,...,...,...
2699,59,55,0.009353
2700,59,56,0.012472
2701,59,57,0.000000
2702,59,58,0.005191


In [19]:
matriz_recomendaciones_long = matriz_recomendaciones_long[matriz_recomendaciones_long['id1'] != matriz_recomendaciones_long['id2']]
matriz_recomendaciones_long

,id1,id2,similitud
1,0,2,0.000000
2,0,3,0.000000
3,0,4,0.000000
4,0,6,0.005578
5,0,8,0.001709
...,...,...,...
2698,59,54,0.000000
2699,59,55,0.009353
2700,59,56,0.012472
2701,59,57,0.000000


In [20]:
matriz_recomendaciones_long = matriz_recomendaciones_long[matriz_recomendaciones_long['id1'] < matriz_recomendaciones_long['id2']]
matriz_recomendaciones_long

,id1,id2,similitud
1,0,2,0.000000
2,0,3,0.000000
3,0,4,0.000000
4,0,6,0.005578
5,0,8,0.001709
...,...,...,...
2546,56,58,0.014804
2547,56,59,0.012472
2598,57,58,0.009789
2599,57,59,0.000000


In [21]:
# Unir las imágenes de 'id2'
matriz_recomendaciones_long = matriz_recomendaciones_long.join(images, on='id2')
matriz_recomendaciones_long

,id1,id2,similitud,image
1,0,2,0.000000,http://books.google.com/books/content?id=AeM0A...
2,0,3,0.000000,http://books.google.com/books/content?id=A6Zzg...
3,0,4,0.000000,http://books.google.com/books/content?id=XE_Ot...
4,0,6,0.005578,http://books.google.com/books/content?id=hrJdI...
5,0,8,0.001709,http://books.google.com/books/content?id=GGNeA...
...,...,...,...,...
2546,56,58,0.014804,http://books.google.com/books/content?id=hdeOj...
2547,56,59,0.012472,http://books.google.com/books/content?id=XiRWX...
2598,57,58,0.009789,http://books.google.com/books/content?id=hdeOj...
2599,57,59,0.000000,http://books.google.com/books/content?id=XiRWX...


In [22]:
# GUARDA LA MATRIZ DE RECOMENDACION A DISCO
##matriz_recomendaciones_long.to_pickle("matriz_recomendaciones_long.pkl")